PAGE ON DEMAND

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time

import pandas as pd

# Definir un User-Agent personalizado
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"

# Configurar las opciones de Chrome
options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={user_agent}')
options.add_argument("--disable-gpu")  # Desactiva la GPU para ahorrar recursos
options.add_argument("--no-sandbox")  # Previene errores en algunos sistemas
options.add_argument("--disable-extensions")  # Desactiva las extensiones para mejorar la velocidad
options.add_argument("--disable-images")  # Desactiva las imágenes para mejorar la velocidad
options.add_argument("--start-maximized")
options.headless = True  # Ejecuta el navegador en modo headless (sin interfaz gráfica)

# Iniciar el navegador con las opciones configuradas
driver = webdriver.Chrome(options=options)

In [ ]:
# Navegar a la página de Pluto.tv
driver.get('https://pluto.tv/latam/on-demand')

# Espera que la página cargue completamente (ajusta según sea necesario)
driver.implicitly_wait(10)

time.sleep(5)

In [ ]:
# Encuentra y haz clic en el span "Películas"
span_peliculas = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//span[text()='Películas']")))
span_peliculas.click()

# Espera a que se cargue la sección de películas (ajusta según sea necesario)
time.sleep(5)

In [ ]:
def process_ul_after_peliculas():
    hrefs = []
    ul_counter = 0  # Contador de ul procesados
    max_scrolls = 50  # Aumentar el límite de desplazamientos
    scrolls_done = 0  # Contador de desplazamientos realizados
    
    try:
        # Encuentra el h3 con el texto "Películas"
        start_h3 = driver.find_element(By.XPATH, "//h3[text()='Películas']")
        print(f"Encontrado h3: Películas")
        
        # Inicia la búsqueda después del h3 "Películas"
        current_element = start_h3
        found_retro = False
        
        while scrolls_done < max_scrolls:
            try:
                ul_element = current_element.find_element(By.XPATH, "./following::ul[1]")  # Encuentra el primer ul después del h3
                
                # Procesa el ul actual
                hrefs.extend(extract_hrefs_from_ul(ul_element))
                ul_counter += 1
                
                print(f"Procesado ul {ul_counter}")
                
                # Actualiza current_element para continuar después del ul procesado
                current_element = ul_element
                
                if found_retro:
                    # Continúa procesando 6 ul más después de encontrar "Retro"
                    if ul_counter >= 6:
                        print("Se procesaron 6 ul adicionales después de 'Retro'. Terminando la extracción.")
                        break
                
                else:
                    # Verifica si hay un h3 con el texto 'Retro' después del current_element
                    try:
                        retro_h3 = driver.find_element(By.XPATH, "//h3[contains(text(), 'Retro')]")
                        print("Encontrado h3 con texto 'Retro'. Continuando con la extracción de 6 ul más.")
                        found_retro = True
                        ul_counter = 0  # Reiniciar el contador para procesar 6 ul más
                    except:
                        pass

                # Mueve el foco al ul actual antes de hacer scroll
                driver.execute_script("arguments[0].scrollIntoView();", ul_element)
                print(f"Foco en ul actual antes de AvPag.")
                
                # Presiona AvPag para continuar cargando contenido
                press_page_down(1)
                scrolls_done += 1
                
            except Exception as e:
                print(f"No se encontraron más ul después de 'Películas' o se alcanzó 'Retro'. Error: {e}")
                break
    
    except Exception as e:
        print(f"Error al encontrar el h3 'Películas': {e}")
    
    return hrefs

# Función para presionar AvPag
def press_page_down(times):
    actions = ActionChains(driver)
    for _ in range(times):
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(2)  # Pausa para permitir la carga del contenido

# Función para extraer hrefs de un ul
def extract_hrefs_from_ul(ul_element):
    hrefs = []
    li_elements = ul_element.find_elements(By.TAG_NAME, 'li')
    for li in li_elements:
        a_tag = li.find_element(By.TAG_NAME, 'a')
        hrefs.append(a_tag.get_attribute('href'))
    return hrefs

# Ejecuta la función para extraer los hrefs
peliculas_hrefs = process_ul_after_peliculas()

# Imprime las URLs extraídas
print(f"Películas URLs: {peliculas_hrefs}")

# Cierra el navegador
driver.quit()


In [ ]:
# Iniciar el navegador con las opciones configuradas
driver = webdriver.Chrome(options=options)

def extract_info_from_li(info_inner_lis):
    rating = ""
    content_type = ""
    seasons_or_duration = ""

    for li in info_inner_lis:
        try:
            # Busca si hay un <span> con la clase 'rating' dentro del <li>
            span = li.find_element(By.TAG_NAME, 'span')
            if 'rating' in span.get_attribute('class'):
                rating = li.text
        except:
            text = li.text.strip()
            if text and 'separator' not in li.get_attribute('class'):
                if 'min' in text or 'Season' in text or 'Available' in text:
                    seasons_or_duration = text
                elif not content_type:
                    content_type = text
                else:
                    seasons_or_duration = text

    return rating, content_type, seasons_or_duration

def check_links_status_with_selenium(links):
    status_results = []
    
    for index, link in enumerate(links):
        try:
            print(f"Accediendo al link {index + 1}/{len(links)}: {link}")
            
            # Navega a la URL
            driver.get(link)
            
            # Espera a que aparezca el div con id 'overlay-container' o la clase 'overlayContainer-0-2-15'
            try:
                overlay_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located(
                        (By.ID, 'overlay-container')
                    )
                )
            except:
                try:
                    overlay_element = WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located(
                            (By.CLASS_NAME, 'overlayContainer-0-2-15')
                        )
                    )
                except Exception as e:
                    print(f"No se encontró el overlay. Saltando al siguiente link. Error: {e}")
                    continue
                
            time.sleep(8)
            
            # TITLE
            try:
                header_element = overlay_element.find_element(By.TAG_NAME, 'header')
                h1_element = header_element.find_element(By.TAG_NAME, 'h1')
                h1_text = h1_element.text
                print(f"Texto del h1: {h1_text}")
            except Exception as e:
                h1_text = f"No se pudo encontrar el h1: {e}"
           
            # DESCRIPTION
            try:
                div_inner = driver.find_element(By.CSS_SELECTOR, 'div.inner')
                p_element = div_inner.find_element(By.TAG_NAME, 'p')
                description_p = p_element.text
            except Exception as e:
                description_p = f"No se pudo encontrar la descripción: {e}"
            print(description_p)

            # INFORMATION
            try:
                info_inner_lis = div_inner.find_elements(By.TAG_NAME, 'li')
                rating, content_type, seasons_or_duration = extract_info_from_li(info_inner_lis)
                
                # RESULTADOS
                print(f"Rating: {rating}")
                print(f"Tipo de Contenido: {content_type}")
                print(f"Temporadas o Duración: {seasons_or_duration}")
            except Exception as e:
                print(f"Error al extraer la información: {e}")
                rating, content_type, seasons_or_duration = "", "", ""

            # CHECK FOR SEASONS
            try:
                season_div = driver.find_element(By.CLASS_NAME, 'season-select')
                select_element = season_div.find_element(By.TAG_NAME, 'select')
                select = Select(select_element)
                
                seasons_info = []
                for option in select.options:
                    season = option.text.strip()
                    print(f"Temporada encontrada: {season}")
                    select.select_by_visible_text(season)
                    time.sleep(3)  
                    
                    episode_elements = driver.find_elements(By.CSS_SELECTOR, 'li.episode-container-atc')
                    for episode in episode_elements:
                        section_detail = episode.find_element(By.CSS_SELECTOR, 'section.episode-details')
                        h3_detail_episode = section_detail.find_element(By.CSS_SELECTOR, 'h3.episode-name-atc').text
                        p_detail_episode = section_detail.find_element(By.CSS_SELECTOR, 'p.episode-description-atc').text
                        
                        try:
                            episode_metadata = episode.find_element(By.CLASS_NAME, 'episode-metadata-atc')
                            episode_numbers = episode_metadata.find_element(By.CLASS_NAME, 'numbers')
                            episode_number = episode_numbers.find_elements(By.TAG_NAME, 'span')[0].text
                            episode_duration = episode_numbers.find_elements(By.TAG_NAME, 'span')[1].text
                        except Exception as e:
                            episode_number = "No disponible"
                            episode_duration = "No disponible"
                            print(f"No se pudieron extraer los datos del episodio: {e}")

                        seasons_info.append({
                            'season': season,
                            'Title episode': h3_detail_episode,
                            'Description episode': p_detail_episode,
                            'Episode number': episode_number,
                            'Duration': episode_duration
                        })
                
                content_type = "Serie"
                print(f"Información de temporadas y capítulos extraída.")
                print(seasons_info)
            except Exception as e:
                content_type = "Película"
                print(f"No se encontró la selección de temporadas. Es probable que sea una película. Error: {e}")
            
            status_results.append({
                'Url': link,
                'Type': content_type,
                'Title': h1_text,
                'Resume': description_p,
                'Rating': rating,
                'Content Type': content_type,
                'Duration': seasons_or_duration,
                'Seasons': seasons_info if content_type == "Serie" else 'No aplica'
            })
            print(f"Información extraída del link {index + 1}/{len(links)}")
        
        except Exception as e:
            status_results.append({
                'url': link,
                'type': 'Error',
                'title': 'Error',
                'description': str(e)
            })
            print(f"Error al acceder al link {index + 1}/{len(links)}: {e}")
    
    return status_results

# Ejecuta la función con las URLs almacenadas en peliculas_hrefs
status_results = check_links_status_with_selenium(peliculas_hrefs)

# Convertir los resultados a un DataFrame de Pandas
df = pd.DataFrame(status_results)

# Exportar el DataFrame a un archivo Excel
excel_path = 'resultados_peliculas_series.xlsx'
df.to_excel(excel_path, index=False, engine='openpyxl')

print(f"Resultados exportados a {excel_path}")

# Cierra el navegador
driver.quit()


PAGE LIVE TV

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time

import pandas as pd

# Definir un User-Agent personalizado
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"

# Configurar las opciones de Chrome
options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={user_agent}')
options.add_argument("--disable-gpu")  # Desactiva la GPU para ahorrar recursos
options.add_argument("--no-sandbox")  # Previene errores en algunos sistemas
options.add_argument("--disable-extensions")  # Desactiva las extensiones para mejorar la velocidad
options.add_argument("--disable-images")  # Desactiva las imágenes para mejorar la velocidad
options.add_argument("--start-maximized")
options.headless = True  # Ejecuta el navegador en modo headless (sin interfaz gráfica)

# Iniciar el navegador con las opciones configuradas
driver = webdriver.Chrome(options=options)

# Navegar a la página de Pluto.tv
driver.get('https://pluto.tv/latam/live-tv')

# Espera que la página cargue completamente (ajusta según sea necesario)
driver.implicitly_wait(5)

time.sleep(5)

# Encuentra y haz clic en el span "Películas"
span_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.textContainer-0-2-122")))
span_button.click()

# Espera a que se cargue la sección de películas (ajusta según sea necesario)
time.sleep(5)

# Esperar a que los divs con la clase deseada estén presentes
divs = driver.find_elements(By.CLASS_NAME, 'channelListItem-0-2-258')

# Iterar sobre cada div y extraer la información
for index, div in enumerate(divs):
    if index > 0:
        # Hacer clic en el div si no es el primero
        div.click()
        time.sleep(2)  # Esperar a que se active el div

    container_data = driver.find_element(By.CSS_SELECTOR, 'div.timelineSkeletonContainer')  

    title_tv = container_data.find_element(By.CSS_SELECTOR, 'span.name-item').text
    print(title_tv)    

# Cerrar el navegador
driver.quit()


CSI: Miami
CSI: Miami


ElementClickInterceptedException: Message: element click intercepted: Element <div class="channelListItem-0-2-258 channel" role="row" aria-rowindex="3" aria-level="2" style="position: absolute; left: 0px; top: 258px; height: 92px; width: 100%;">...</div> is not clickable at point (1100, 403). Other element would receive the click: <div class="inner">...</div>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00007FF641249642+30946]
	(No symbol) [0x00007FF6411FE3D9]
	(No symbol) [0x00007FF6410F6FDA]
	(No symbol) [0x00007FF64114FEEE]
	(No symbol) [0x00007FF64114D962]
	(No symbol) [0x00007FF64114AE7B]
	(No symbol) [0x00007FF64114A095]
	(No symbol) [0x00007FF64113C271]
	(No symbol) [0x00007FF64116CA6A]
	(No symbol) [0x00007FF64113BBB6]
	(No symbol) [0x00007FF64116CC80]
	(No symbol) [0x00007FF64118B041]
	(No symbol) [0x00007FF64116C813]
	(No symbol) [0x00007FF64113A6E5]
	(No symbol) [0x00007FF64113B021]
	GetHandleVerifier [0x00007FF64137F84D+1301229]
	GetHandleVerifi